In [1]:
import os
import xarray as xr
import numpy as np
os.getcwd()

/N/u/amulla/Quartz/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


'/geode2/home/u010/amulla/Quartz/Desktop/hourly_precipitation_analysis'

In [2]:
%cd /N/project/Zli_lab/ERA5_land_data
curr_data_path = os.getcwd()
curr_data_path

/N/project/Zli_lab/ERA5_land_data


'/N/project/Zli_lab/ERA5_land_data'

In [3]:
def read_data(s_dir, month, year):
    path = os.path.join(s_dir, f"Hourly_Total_Precipitation_T_{str(month).zfill(2)}{year}.nc")
    ds = xr.open_dataset(path)
    return ds

In [9]:
ex = read_data('../ERA5_land_data_recalculated/JJA_prev_val', 6,1979)
np.where(np.arange(len(ex.tp.data))%24==2)
# np.where(np.arange(len(data))%24==2)[0]

(array([  2,  26,  50,  74,  98, 122, 146, 170, 194, 218, 242, 266, 290,
        314, 338, 362, 386, 410, 434, 458, 482, 506, 530, 554, 578, 602,
        626, 650, 674, 698]),)

# JJA

In [ ]:
5-6-7-8

In [5]:
def recalculate(data):
    nan_mask = np.isnan(data)
    binary_mask = np.where(nan_mask, 0, 1)
    numeric_indices = np.where(binary_mask == 1)[0]
    if len(numeric_indices) == 0:
        differenced_data = data[1:]
    elif len(numeric_indices) == len(data):
        differenced_data = np.zeros_like(data[1:])
        indices_01utc = np.where(np.arange(len(data))%24==2)[0]
        indices_allutc = np.where(np.arange(len(data))%24!=2)[0][1:]
        differenced_data[1::24] = data[indices_01utc]
        differenced_data[indices_allutc - 1] = np.diff(data)[indices_allutc - 1]
    else:
        print("Error Encountered...")
    return differenced_data

In [6]:
for year in range(1979,2024):
    ds_6 = read_data(curr_data_path, 6, year)
    ds_7 = read_data(curr_data_path, 7, year)
    your_existing_dataarray = ds_6.tp[-1]
    print(ds_6.tp[-1].time.data)
    new_ds = xr.Dataset({'tp': your_existing_dataarray})
    combined_ds = xr.concat([new_ds, ds_7], dim='time')
    combined_ds = combined_ds.transpose('time' , 'latitude', 'longitude')
    combined_ds.to_netcdf(f"../ERA5_land_data_recalculated/JJA/Hourly_Total_Precipitation_T_{str(7).zfill(2)}{year}.nc")
    result = xr.apply_ufunc(
            recalculate,
            combined_ds,
            input_core_dims=[['time']],
            vectorize=True,
            output_core_dims=[['time_recalc']],
            output_dtypes=[np.float64],
            dask='parallelized',
        )
    result = result.transpose('time_recalc' ,'latitude', 'longitude')
    
    time = combined_ds.time[1:]
    latitude = result['latitude']
    longitude = result['longitude']

    final_result = xr.Dataset(coords={'latitude': latitude, 'longitude': longitude, 'time': time})

    final_result['tp'] = (('time', 'latitude', 'longitude'), np.nan * np.ones((len(time), len(latitude), len(longitude))))

    final_result['tp'][:,:,:] = result['tp'][:,:,:].data
    final_result.to_netcdf(f"../ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_{str(7).zfill(2)}{year}.nc")
    print(year)


1979-06-30T23:00:00.000000000
1979
1980-06-30T23:00:00.000000000
1980
1981-06-30T23:00:00.000000000
1981
1982-06-30T23:00:00.000000000
1982
1983-06-30T23:00:00.000000000
1983
1984-06-30T23:00:00.000000000
1984
1985-06-30T23:00:00.000000000
1985
1986-06-30T23:00:00.000000000
1986
1987-06-30T23:00:00.000000000
1987
1988-06-30T23:00:00.000000000
1988
1989-06-30T23:00:00.000000000
1989
1990-06-30T23:00:00.000000000
1990
1991-06-30T23:00:00.000000000
1991
1992-06-30T23:00:00.000000000
1992
1993-06-30T23:00:00.000000000
1993
1994-06-30T23:00:00.000000000
1994
1995-06-30T23:00:00.000000000
1995
1996-06-30T23:00:00.000000000
1996
1997-06-30T23:00:00.000000000
1997
1998-06-30T23:00:00.000000000
1998
1999-06-30T23:00:00.000000000
1999
2000-06-30T23:00:00.000000000
2000
2001-06-30T23:00:00.000000000
2001
2002-06-30T23:00:00.000000000
2002
2003-06-30T23:00:00.000000000
2003
2004-06-30T23:00:00.000000000
2004
2005-06-30T23:00:00.000000000
2005
2006-06-30T23:00:00.000000000
2006
2007-06-30T23:00:00.

In [7]:
for year in range(1979,2024):
    ds_7 = read_data(curr_data_path, 7, year)
    ds_8 = read_data(curr_data_path, 8, year)
    your_existing_dataarray = ds_7.tp[-1]
    new_ds = xr.Dataset({'tp': your_existing_dataarray})
    combined_ds = xr.concat([new_ds, ds_8], dim='time')
    combined_ds = combined_ds.transpose('time' , 'latitude', 'longitude')
    combined_ds.to_netcdf(f"../ERA5_land_data_recalculated/JJA/Hourly_Total_Precipitation_T_{str(8).zfill(2)}{year}.nc")
    result = xr.apply_ufunc(
            recalculate,
            combined_ds,
            input_core_dims=[['time']],
            vectorize=True,
            output_core_dims=[['time_recalc']],
            output_dtypes=[np.float64],
            dask='parallelized',
        )
    result = result.transpose('time_recalc' ,'latitude', 'longitude')
    
    time = combined_ds.time[1:]
    latitude = result['latitude']
    longitude = result['longitude']

    final_result = xr.Dataset(coords={'latitude': latitude, 'longitude': longitude, 'time': time})

    final_result['tp'] = (('time', 'latitude', 'longitude'), np.nan * np.ones((len(time), len(latitude), len(longitude))))

    final_result['tp'][:,:,:] = result['tp'][:,:,:].data
    final_result.to_netcdf(f"../ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_{str(8).zfill(2)}{year}.nc")
    print(year)


1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [222]:
# Perform temporal aggregation by calculating percentile and then nansum or directly nansum for total accumulation

In [4]:
yearly_data = []
for year in range(1979,1980):
    for month in range(9, 10):
        ds = read_data(curr_data_path, month, year)
        if 'expver' in ds.dims:
            ds = ds.sel(expver=1).combine_first(ds.sel(expver=5))
            yearly_data.append(ds)
        else:
            yearly_data.append(ds)

# MAM

In [ ]:
2-3-4-5

In [4]:
def recalculate(data):
    nan_mask = np.isnan(data)
    binary_mask = np.where(nan_mask, 0, 1)
    numeric_indices = np.where(binary_mask == 1)[0]
    if len(numeric_indices) == 0:
        differenced_data = data[1:]
    elif len(numeric_indices) == len(data):
        differenced_data = np.zeros_like(data[1:])
        indices_01utc = np.where(np.arange(len(data))%24==2)[0]
        indices_allutc = np.where(np.arange(len(data))%24!=2)[0][1:]
        differenced_data[1::24] = data[indices_01utc]
        differenced_data[indices_allutc - 1] = np.diff(data)[indices_allutc - 1]
    else:
        print("Error Encountered...")
    return differenced_data

In [73]:
for year in range(1979,2024):
    ds_2 = read_data(curr_data_path, 2, year)
    ds_3 = read_data(curr_data_path, 3, year)
    your_existing_dataarray = ds_2.tp[-1]
    print(ds_2.tp[-1].time.data)
    new_ds = xr.Dataset({'tp': your_existing_dataarray})
    combined_ds = xr.concat([new_ds, ds_3], dim='time')
    combined_ds = combined_ds.transpose('time' , 'latitude', 'longitude')
    combined_ds.to_netcdf(f"../ERA5_land_data_recalculated/MAM_prev_val/Hourly_Total_Precipitation_T_{str(3).zfill(2)}{year}.nc")
    result = xr.apply_ufunc(
            recalculate,
            combined_ds,
            input_core_dims=[['time']],
            vectorize=True,
            output_core_dims=[['time_recalc']],
            output_dtypes=[np.float64],
            dask='parallelized',
        )
    result = result.transpose('time_recalc' ,'latitude', 'longitude')
    
    time = combined_ds.time[1:]
    latitude = result['latitude']
    longitude = result['longitude']

    final_result = xr.Dataset(coords={'latitude': latitude, 'longitude': longitude, 'time': time})

    final_result['tp'] = (('time', 'latitude', 'longitude'), np.nan * np.ones((len(time), len(latitude), len(longitude))))

    final_result['tp'][:,:,:] = result['tp'][:,:,:].data
    final_result.to_netcdf(f"../ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_{str(3).zfill(2)}{year}.nc")
    print(year)


1979-02-28T23:00:00.000000000
1979
1980-02-29T23:00:00.000000000
1980
1981-02-28T23:00:00.000000000
1981
1982-02-28T23:00:00.000000000
1982
1983-02-28T23:00:00.000000000
1983
1984-02-29T23:00:00.000000000
1984
1985-02-28T23:00:00.000000000
1985
1986-02-28T23:00:00.000000000
1986
1987-02-28T23:00:00.000000000
1987
1988-02-29T23:00:00.000000000
1988
1989-02-28T23:00:00.000000000
1989
1990-02-28T23:00:00.000000000
1990
1991-02-28T23:00:00.000000000
1991
1992-02-29T23:00:00.000000000
1992
1993-02-28T23:00:00.000000000
1993
1994-02-28T23:00:00.000000000
1994
1995-02-28T23:00:00.000000000
1995
1996-02-29T23:00:00.000000000
1996
1997-02-28T23:00:00.000000000
1997
1998-02-28T23:00:00.000000000
1998
1999-02-28T23:00:00.000000000
1999
2000-02-29T23:00:00.000000000
2000
2001-02-28T23:00:00.000000000
2001
2002-02-28T23:00:00.000000000
2002
2003-02-28T23:00:00.000000000
2003
2004-02-29T23:00:00.000000000
2004
2005-02-28T23:00:00.000000000
2005
2006-02-28T23:00:00.000000000
2006
2007-02-28T23:00:00.

In [5]:
for year in range(1979,2024):
    ds_3 = read_data(curr_data_path, 3, year)
    ds_4 = read_data(curr_data_path, 4, year)
    your_existing_dataarray = ds_3.tp[-1]
    print(ds_3.tp[-1].time.data)
    new_ds = xr.Dataset({'tp': your_existing_dataarray})
    combined_ds = xr.concat([new_ds, ds_4], dim='time')
    combined_ds = combined_ds.transpose('time' , 'latitude', 'longitude')
    combined_ds.to_netcdf(f"../ERA5_land_data_recalculated/MAM_prev_val/Hourly_Total_Precipitation_T_{str(4).zfill(2)}{year}.nc")
    result = xr.apply_ufunc(
            recalculate,
            combined_ds,
            input_core_dims=[['time']],
            vectorize=True,
            output_core_dims=[['time_recalc']],
            output_dtypes=[np.float64],
            dask='parallelized',
        )
    result = result.transpose('time_recalc' ,'latitude', 'longitude')
    
    time = combined_ds.time[1:]
    latitude = result['latitude']
    longitude = result['longitude']

    final_result = xr.Dataset(coords={'latitude': latitude, 'longitude': longitude, 'time': time})

    final_result['tp'] = (('time', 'latitude', 'longitude'), np.nan * np.ones((len(time), len(latitude), len(longitude))))

    final_result['tp'][:,:,:] = result['tp'][:,:,:].data
    final_result.to_netcdf(f"../ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_{str(4).zfill(2)}{year}.nc")
    print(year)

1979-03-31T23:00:00.000000000
1979
1980-03-31T23:00:00.000000000
1980
1981-03-31T23:00:00.000000000
1981
1982-03-31T23:00:00.000000000
1982
1983-03-31T23:00:00.000000000
1983
1984-03-31T23:00:00.000000000
1984
1985-03-31T23:00:00.000000000
1985
1986-03-31T23:00:00.000000000
1986
1987-03-31T23:00:00.000000000
1987
1988-03-31T23:00:00.000000000
1988
1989-03-31T23:00:00.000000000
1989
1990-03-31T23:00:00.000000000
1990
1991-03-31T23:00:00.000000000
1991
1992-03-31T23:00:00.000000000
1992
1993-03-31T23:00:00.000000000
1993
1994-03-31T23:00:00.000000000
1994
1995-03-31T23:00:00.000000000
1995
1996-03-31T23:00:00.000000000
1996
1997-03-31T23:00:00.000000000
1997
1998-03-31T23:00:00.000000000
1998
1999-03-31T23:00:00.000000000
1999
2000-03-31T23:00:00.000000000
2000
2001-03-31T23:00:00.000000000
2001
2002-03-31T23:00:00.000000000
2002
2003-03-31T23:00:00.000000000
2003
2004-03-31T23:00:00.000000000
2004
2005-03-31T23:00:00.000000000
2005
2006-03-31T23:00:00.000000000
2006
2007-03-31T23:00:00.

In [5]:
for year in range(1979,2024):
    ds_4 = read_data(curr_data_path, 4, year)
    ds_5 = read_data(curr_data_path, 5, year)
    your_existing_dataarray = ds_4.tp[-1]
    print(ds_4.tp[-1].time.data)
    new_ds = xr.Dataset({'tp': your_existing_dataarray})
    combined_ds = xr.concat([new_ds, ds_5], dim='time')
    combined_ds = combined_ds.transpose('time' , 'latitude', 'longitude')
    combined_ds.to_netcdf(f"../ERA5_land_data_recalculated/MAM_prev_val/Hourly_Total_Precipitation_T_{str(5).zfill(2)}{year}.nc")
    result = xr.apply_ufunc(
            recalculate,
            combined_ds,
            input_core_dims=[['time']],
            vectorize=True,
            output_core_dims=[['time_recalc']],
            output_dtypes=[np.float64],
            dask='parallelized',
        )
    result = result.transpose('time_recalc' ,'latitude', 'longitude')
    
    time = combined_ds.time[1:]
    latitude = result['latitude']
    longitude = result['longitude']

    final_result = xr.Dataset(coords={'latitude': latitude, 'longitude': longitude, 'time': time})

    final_result['tp'] = (('time', 'latitude', 'longitude'), np.nan * np.ones((len(time), len(latitude), len(longitude))))

    final_result['tp'][:,:,:] = result['tp'][:,:,:].data
    final_result.to_netcdf(f"../ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_{str(5).zfill(2)}{year}.nc")
    print(year)

1979-04-30T23:00:00.000000000
1979
1980-04-30T23:00:00.000000000
1980
1981-04-30T23:00:00.000000000
1981
1982-04-30T23:00:00.000000000
1982
1983-04-30T23:00:00.000000000
1983
1984-04-30T23:00:00.000000000
1984
1985-04-30T23:00:00.000000000
1985
1986-04-30T23:00:00.000000000
1986
1987-04-30T23:00:00.000000000
1987
1988-04-30T23:00:00.000000000
1988
1989-04-30T23:00:00.000000000
1989
1990-04-30T23:00:00.000000000
1990
1991-04-30T23:00:00.000000000
1991
1992-04-30T23:00:00.000000000
1992
1993-04-30T23:00:00.000000000
1993
1994-04-30T23:00:00.000000000
1994
1995-04-30T23:00:00.000000000
1995
1996-04-30T23:00:00.000000000
1996
1997-04-30T23:00:00.000000000
1997
1998-04-30T23:00:00.000000000
1998
1999-04-30T23:00:00.000000000
1999
2000-04-30T23:00:00.000000000
2000
2001-04-30T23:00:00.000000000
2001
2002-04-30T23:00:00.000000000
2002
2003-04-30T23:00:00.000000000
2003
2004-04-30T23:00:00.000000000
2004
2005-04-30T23:00:00.000000000
2005
2006-04-30T23:00:00.000000000
2006
2007-04-30T23:00:00.

# SON

In [ ]:
8-9-10-11

In [4]:
def recalculate(data):
    nan_mask = np.isnan(data)
    binary_mask = np.where(nan_mask, 0, 1)
    numeric_indices = np.where(binary_mask == 1)[0]
    if len(numeric_indices) == 0:
        differenced_data = data[1:]
    elif len(numeric_indices) == len(data):
        differenced_data = np.zeros_like(data[1:])
        indices_01utc = np.where(np.arange(len(data))%24==2)[0]
        indices_allutc = np.where(np.arange(len(data))%24!=2)[0][1:]
        differenced_data[1::24] = data[indices_01utc]
        differenced_data[indices_allutc - 1] = np.diff(data)[indices_allutc - 1]
    else:
        print("Error Encountered...")
    return differenced_data

In [5]:
for year in range(1979,2024):
    ds_8 = read_data(curr_data_path, 8, year)
    ds_9 = read_data(curr_data_path, 9, year)
    your_existing_dataarray = ds_8.tp[-1]
    print(ds_8.tp[-1].time.data)
    new_ds = xr.Dataset({'tp': your_existing_dataarray})
    combined_ds = xr.concat([new_ds, ds_9], dim='time')
    combined_ds = combined_ds.transpose('time' , 'latitude', 'longitude')
    combined_ds.to_netcdf(f"../ERA5_land_data_recalculated/SON_prev_val/Hourly_Total_Precipitation_T_{str(9).zfill(2)}{year}.nc")
    result = xr.apply_ufunc(
            recalculate,
            combined_ds,
            input_core_dims=[['time']],
            vectorize=True,
            output_core_dims=[['time_recalc']],
            output_dtypes=[np.float64],
            dask='parallelized',
        )
    result = result.transpose('time_recalc' ,'latitude', 'longitude')
    
    time = combined_ds.time[1:]
    latitude = result['latitude']
    longitude = result['longitude']

    final_result = xr.Dataset(coords={'latitude': latitude, 'longitude': longitude, 'time': time})

    final_result['tp'] = (('time', 'latitude', 'longitude'), np.nan * np.ones((len(time), len(latitude), len(longitude))))

    final_result['tp'][:,:,:] = result['tp'][:,:,:].data
    final_result.to_netcdf(f"../ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_{str(9).zfill(2)}{year}.nc")
    print(year)


1979-08-31T23:00:00.000000000
1979
1980-08-31T23:00:00.000000000
1980
1981-08-31T23:00:00.000000000
1981
1982-08-31T23:00:00.000000000
1982
1983-08-31T23:00:00.000000000
1983
1984-08-31T23:00:00.000000000
1984
1985-08-31T23:00:00.000000000
1985
1986-08-31T23:00:00.000000000
1986
1987-08-31T23:00:00.000000000
1987
1988-08-31T23:00:00.000000000
1988
1989-08-31T23:00:00.000000000
1989
1990-08-31T23:00:00.000000000
1990
1991-08-31T23:00:00.000000000
1991
1992-08-31T23:00:00.000000000
1992
1993-08-31T23:00:00.000000000
1993
1994-08-31T23:00:00.000000000
1994
1995-08-31T23:00:00.000000000
1995
1996-08-31T23:00:00.000000000
1996
1997-08-31T23:00:00.000000000
1997
1998-08-31T23:00:00.000000000
1998
1999-08-31T23:00:00.000000000
1999
2000-08-31T23:00:00.000000000
2000
2001-08-31T23:00:00.000000000
2001
2002-08-31T23:00:00.000000000
2002
2003-08-31T23:00:00.000000000
2003
2004-08-31T23:00:00.000000000
2004
2005-08-31T23:00:00.000000000
2005
2006-08-31T23:00:00.000000000
2006
2007-08-31T23:00:00.

In [6]:
for year in range(1979,2024):
    ds_9 = read_data(curr_data_path, 9, year)
    ds_10 = read_data(curr_data_path, 10, year)
    your_existing_dataarray = ds_9.tp[-1]
    print(ds_9.tp[-1].time.data)
    new_ds = xr.Dataset({'tp': your_existing_dataarray})
    combined_ds = xr.concat([new_ds, ds_10], dim='time')
    combined_ds = combined_ds.transpose('time' , 'latitude', 'longitude')
    combined_ds.to_netcdf(f"../ERA5_land_data_recalculated/SON_prev_val/Hourly_Total_Precipitation_T_{str(10).zfill(2)}{year}.nc")
    result = xr.apply_ufunc(
            recalculate,
            combined_ds,
            input_core_dims=[['time']],
            vectorize=True,
            output_core_dims=[['time_recalc']],
            output_dtypes=[np.float64],
            dask='parallelized',
        )
    result = result.transpose('time_recalc' ,'latitude', 'longitude')
    
    time = combined_ds.time[1:]
    latitude = result['latitude']
    longitude = result['longitude']

    final_result = xr.Dataset(coords={'latitude': latitude, 'longitude': longitude, 'time': time})

    final_result['tp'] = (('time', 'latitude', 'longitude'), np.nan * np.ones((len(time), len(latitude), len(longitude))))

    final_result['tp'][:,:,:] = result['tp'][:,:,:].data
    final_result.to_netcdf(f"../ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_{str(10).zfill(2)}{year}.nc")
    print(year)


1979-09-30T23:00:00.000000000
1979
1980-09-30T23:00:00.000000000
1980
1981-09-30T23:00:00.000000000
1981
1982-09-30T23:00:00.000000000
1982
1983-09-30T23:00:00.000000000
1983
1984-09-30T23:00:00.000000000
1984
1985-09-30T23:00:00.000000000
1985
1986-09-30T23:00:00.000000000
1986
1987-09-30T23:00:00.000000000
1987
1988-09-30T23:00:00.000000000
1988
1989-09-30T23:00:00.000000000
1989
1990-09-30T23:00:00.000000000
1990
1991-09-30T23:00:00.000000000
1991
1992-09-30T23:00:00.000000000
1992
1993-09-30T23:00:00.000000000
1993
1994-09-30T23:00:00.000000000
1994
1995-09-30T23:00:00.000000000
1995
1996-09-30T23:00:00.000000000
1996
1997-09-30T23:00:00.000000000
1997
1998-09-30T23:00:00.000000000
1998
1999-09-30T23:00:00.000000000
1999
2000-09-30T23:00:00.000000000
2000
2001-09-30T23:00:00.000000000
2001
2002-09-30T23:00:00.000000000
2002
2003-09-30T23:00:00.000000000
2003
2004-09-30T23:00:00.000000000
2004
2005-09-30T23:00:00.000000000
2005
2006-09-30T23:00:00.000000000
2006
2007-09-30T23:00:00.

In [8]:
for year in range(1979,2024):
    ds_10 = read_data(curr_data_path, 10, year)
    ds_11 = read_data(curr_data_path, 11, year)
    if 'expver' in ds_11.dims:
            # Remove the 'expver' dimension using isel
            ds_11 = ds_11.sel(expver=1).combine_first(ds_11.sel(expver=5))
    your_existing_dataarray = ds_10.tp[-1]
    print(ds_10.tp[-1].time.data)
    new_ds = xr.Dataset({'tp': your_existing_dataarray})
    combined_ds = xr.concat([new_ds, ds_11], dim='time')
    combined_ds = combined_ds.transpose('time' , 'latitude', 'longitude')
    combined_ds.to_netcdf(f"../ERA5_land_data_recalculated/SON_prev_val/Hourly_Total_Precipitation_T_{str(11).zfill(2)}{year}.nc")
    result = xr.apply_ufunc(
            recalculate,
            combined_ds,
            input_core_dims=[['time']],
            vectorize=True,
            output_core_dims=[['time_recalc']],
            output_dtypes=[np.float64],
            dask='parallelized',
        )
    result = result.transpose('time_recalc' ,'latitude', 'longitude')
    
    time = combined_ds.time[1:]
    latitude = result['latitude']
    longitude = result['longitude']

    final_result = xr.Dataset(coords={'latitude': latitude, 'longitude': longitude, 'time': time})

    final_result['tp'] = (('time', 'latitude', 'longitude'), np.nan * np.ones((len(time), len(latitude), len(longitude))))

    final_result['tp'][:,:,:] = result['tp'][:,:,:].data
    final_result.to_netcdf(f"../ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_{str(11).zfill(2)}{year}.nc")
    print(year)


2023-10-31T23:00:00.000000000
2023


# DJF

In [ ]:
11-12-1-2

In [5]:
def recalculate(data):
    nan_mask = np.isnan(data)
    binary_mask = np.where(nan_mask, 0, 1)
    numeric_indices = np.where(binary_mask == 1)[0]
    if len(numeric_indices) == 0:
        differenced_data = data[1:]
    elif len(numeric_indices) == len(data):
        differenced_data = np.zeros_like(data[1:])
        indices_01utc = np.where(np.arange(len(data))%24==2)[0]
        indices_allutc = np.where(np.arange(len(data))%24!=2)[0][1:]
        differenced_data[1::24] = data[indices_01utc]
        differenced_data[indices_allutc - 1] = np.diff(data)[indices_allutc - 1]
    else:
        print("Error Encountered...")
    return differenced_data

In [6]:
for year in range(1978,2023):
    ds_11 = read_data(curr_data_path, 11, year)
    ds_12 = read_data(curr_data_path, 12, year)
    if 'expver' in ds_11.dims:
            # Remove the 'expver' dimension using isel
            ds_11 = ds_11.sel(expver=1).combine_first(ds_11.sel(expver=5))
    your_existing_dataarray = ds_11.tp[-1]
    print(ds_11.tp[-1].time.data)
    new_ds = xr.Dataset({'tp': your_existing_dataarray})
    combined_ds = xr.concat([new_ds, ds_12], dim='time')
    combined_ds = combined_ds.transpose('time' , 'latitude', 'longitude')
    combined_ds.to_netcdf(f"../ERA5_land_data_recalculated/DJF_prev_val/Hourly_Total_Precipitation_T_{str(12).zfill(2)}{year}.nc")
    result = xr.apply_ufunc(
            recalculate,
            combined_ds,
            input_core_dims=[['time']],
            vectorize=True,
            output_core_dims=[['time_recalc']],
            output_dtypes=[np.float64],
            dask='parallelized',
        )
    result = result.transpose('time_recalc' ,'latitude', 'longitude')
    
    time = combined_ds.time[1:]
    latitude = result['latitude']
    longitude = result['longitude']

    final_result = xr.Dataset(coords={'latitude': latitude, 'longitude': longitude, 'time': time})

    final_result['tp'] = (('time', 'latitude', 'longitude'), np.nan * np.ones((len(time), len(latitude), len(longitude))))

    final_result['tp'][:,:,:] = result['tp'][:,:,:].data
    final_result.to_netcdf(f"../ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_{str(12).zfill(2)}{year}.nc")
    print(year)

1978-11-30T23:00:00.000000000
1978
1979-11-30T23:00:00.000000000
1979
1980-11-30T23:00:00.000000000
1980
1981-11-30T23:00:00.000000000
1981
1982-11-30T23:00:00.000000000
1982
1983-11-30T23:00:00.000000000
1983
1984-11-30T23:00:00.000000000
1984
1985-11-30T23:00:00.000000000
1985
1986-11-30T23:00:00.000000000
1986
1987-11-30T23:00:00.000000000
1987
1988-11-30T23:00:00.000000000
1988
1989-11-30T23:00:00.000000000
1989
1990-11-30T23:00:00.000000000
1990
1991-11-30T23:00:00.000000000
1991
1992-11-30T23:00:00.000000000
1992
1993-11-30T23:00:00.000000000
1993
1994-11-30T23:00:00.000000000
1994
1995-11-30T23:00:00.000000000
1995
1996-11-30T23:00:00.000000000
1996
1997-11-30T23:00:00.000000000
1997
1998-11-30T23:00:00.000000000
1998
1999-11-30T23:00:00.000000000
1999
2000-11-30T23:00:00.000000000
2000
2001-11-30T23:00:00.000000000
2001
2002-11-30T23:00:00.000000000
2002
2003-11-30T23:00:00.000000000
2003
2004-11-30T23:00:00.000000000
2004
2005-11-30T23:00:00.000000000
2005
2006-11-30T23:00:00.

In [9]:
for year in range(1978,2023):
    ds_12 = read_data(curr_data_path, 12, year)
    ds_1 = read_data(curr_data_path, 1, year+1)
    if 'expver' in ds_12.dims:
            # Remove the 'expver' dimension using isel
            ds_12 = ds_12.sel(expver=1).combine_first(ds_12.sel(expver=5))
    your_existing_dataarray = ds_12.tp[-1]
    print(ds_12.tp[-1].time.data)
    new_ds = xr.Dataset({'tp': your_existing_dataarray})
    combined_ds = xr.concat([new_ds, ds_1], dim='time')
    combined_ds = combined_ds.transpose('time' , 'latitude', 'longitude')
    combined_ds.to_netcdf(f"../ERA5_land_data_recalculated/DJF_prev_val/Hourly_Total_Precipitation_T_{str(1).zfill(2)}{year+1}.nc")
    result = xr.apply_ufunc(
            recalculate,
            combined_ds,
            input_core_dims=[['time']],
            vectorize=True,
            output_core_dims=[['time_recalc']],
            output_dtypes=[np.float64],
            dask='parallelized',
        )
    result = result.transpose('time_recalc' ,'latitude', 'longitude')
    
    time = combined_ds.time[1:]
    latitude = result['latitude']
    longitude = result['longitude']

    final_result = xr.Dataset(coords={'latitude': latitude, 'longitude': longitude, 'time': time})

    final_result['tp'] = (('time', 'latitude', 'longitude'), np.nan * np.ones((len(time), len(latitude), len(longitude))))

    final_result['tp'][:,:,:] = result['tp'][:,:,:].data
    final_result.to_netcdf(f"../ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_{str(1).zfill(2)}{year+1}.nc")
    print(year)

1978-12-31T23:00:00.000000000
1978
1979-12-31T23:00:00.000000000
1979
1980-12-31T23:00:00.000000000
1980
1981-12-31T23:00:00.000000000
1981
1982-12-31T23:00:00.000000000
1982
1983-12-31T23:00:00.000000000
1983
1984-12-31T23:00:00.000000000
1984
1985-12-31T23:00:00.000000000
1985
1986-12-31T23:00:00.000000000
1986
1987-12-31T23:00:00.000000000
1987
1988-12-31T23:00:00.000000000
1988
1989-12-31T23:00:00.000000000
1989
1990-12-31T23:00:00.000000000
1990
1991-12-31T23:00:00.000000000
1991
1992-12-31T23:00:00.000000000
1992
1993-12-31T23:00:00.000000000
1993
1994-12-31T23:00:00.000000000
1994
1995-12-31T23:00:00.000000000
1995
1996-12-31T23:00:00.000000000
1996
1997-12-31T23:00:00.000000000
1997
1998-12-31T23:00:00.000000000
1998
1999-12-31T23:00:00.000000000
1999
2000-12-31T23:00:00.000000000
2000
2001-12-31T23:00:00.000000000
2001
2002-12-31T23:00:00.000000000
2002
2003-12-31T23:00:00.000000000
2003
2004-12-31T23:00:00.000000000
2004
2005-12-31T23:00:00.000000000
2005
2006-12-31T23:00:00.

In [8]:
for year in range(2023,2024):
    ds_1 = read_data(curr_data_path, 1, year)
    ds_2 = read_data(curr_data_path, 2, year)
    your_existing_dataarray = ds_1.tp[-1]
    print(ds_1.tp[-1].time.data)
    new_ds = xr.Dataset({'tp': your_existing_dataarray})
    combined_ds = xr.concat([new_ds, ds_2], dim='time')
    combined_ds = combined_ds.transpose('time' , 'latitude', 'longitude')
    combined_ds.to_netcdf(f"../ERA5_land_data_recalculated/DJF_prev_val/Hourly_Total_Precipitation_T_{str(2).zfill(2)}{year}.nc")
    result = xr.apply_ufunc(
            recalculate,
            combined_ds,
            input_core_dims=[['time']],
            vectorize=True,
            output_core_dims=[['time_recalc']],
            output_dtypes=[np.float64],
            dask='parallelized',
        )
    result = result.transpose('time_recalc' ,'latitude', 'longitude')
    
    time = combined_ds.time[1:]
    latitude = result['latitude']
    longitude = result['longitude']

    final_result = xr.Dataset(coords={'latitude': latitude, 'longitude': longitude, 'time': time})

    final_result['tp'] = (('time', 'latitude', 'longitude'), np.nan * np.ones((len(time), len(latitude), len(longitude))))

    final_result['tp'][:,:,:] = result['tp'][:,:,:].data
    final_result.to_netcdf(f"../ERA5_land_data_recalculated/Hourly_Total_Precipitation_T_{str(2).zfill(2)}{year}.nc")
    print(year)

2023-01-31T23:00:00.000000000
2023


In [13]:
# ds_1178 = read_data(curr_data_path,11,1978)
# ds_1278 = read_data(curr_data_path,12,1978)
# start_time  = len(ds_1178.tp)-24
# end_time = len(ds_1178.tp)
# your_existing_dataarray = ds_1178.tp[-1]
# new_ds = xr.Dataset({'tp': your_existing_dataarray})
# combined_ds = xr.concat([new_ds, ds_1278], dim='time')
# combined_ds = combined_ds.transpose('time' , 'latitude', 'longitude')

# def concat(prev_month, curr_month, prev_year, curr_year):
#     prev_ds = read_data(curr_data_path, prev_month, prev_year)
#     curr_ds = read_data(curr_data_path, curr_month, curr_year)
#     to_append = prev_ds.tp[-1]
#     new_ds = xr.Dataset({'tp': to_append})
#     combined_ds = xr.concat([new_ds, curr_ds], dim='time')
#     combined_ds = combined_ds.transpose('time' , 'latitude', 'longitude')
#     return combined_ds

# d78 = concat(11,12,1978,1978)
# j79 = read_data(curr_data_path,1,1979)
# f79 = read_data(curr_data_path,2,1979)

# con = xr.concat([d78,j79,f79], dim="time")